In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, hinge_loss
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
url = "/content/drive/MyDrive/SEM6/DMW/Prac/C2/Group_A1/letter-recognition.data"

col = ['lettr', 'x-box', 'y-box', 'width', 'high', 'onpix', 'x-bar', 'y-bar', 'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 'x-ege', 'xegvy', 'y-ege', 'yegvx']

irisdata = pd.read_csv(url, names=col)

x = irisdata.drop('lettr', axis=1)
y = irisdata['lettr']

In [ ]:
irisdata.head()

,lettr,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [ ]:
alph = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'I':8, 'J':9, 'K':10, 'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18, 'T':19, 'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25}

In [ ]:
num = round(len(irisdata)*0.8)
train, test = irisdata[:num], irisdata[num:]

In [ ]:
## train should be a dataframe
def kt_svm(train, N, k, q, n2, ker, deg=2):
  ## taking N random samples from train
  random_n = train.sample(N)
  ## training with SVMC
  temp_x = random_n.drop('lettr', axis=1)
  temp_y = random_n['lettr']
  
  if ker == "linear":
    temp_clf = SVC(kernel = ker)
  elif ker == "rbf":
    temp_clf = SVC(kernel = ker, random_state = 0)
  elif ker == "sigmoid":
    temp_clf = SVC(kernel = ker)
  elif ker == "poly":
    temp_clf = SVC(kernel = ker, degree = deg)

  temp_clf.fit(temp_x, temp_y)
  ## let i = 0
  i = 0
  while True:
    ## set N_i and t
    N_i, t = 0, 1
    ## take one random sample from train
    one = train.sample()
    N_i = N_i + 1

    df_store = []
    while True:
      ## take another sample
      ano = train.sample()
      ncpt = 0
      ## calculate ratio
      one_x = one.drop('lettr', axis=1)
      one_y = one['lettr']
      one_i = alph[one_y.values[0]]
      one_dec = temp_clf.decision_function(one_x)
      # one_pred = one_dec[0, one_i]
      one_pred = max(one_dec[0])

      ano_x = ano.drop('lettr', axis=1)
      ano_y = ano['lettr']
      ano_i = alph[ano_y.values[0]]
      ano_dec = temp_clf.decision_function(ano_x)
      # ano_pred = ano_dec[0, ano_i]
      ano_pred = max(ano_dec[0])

      alfa = np.exp(np.log2(abs(ano_pred/one_pred)))
      ## conditions
      if round(alfa, 2) == 1 or alfa < 1 or ncpt == n2:
        ## accept second sample
        df_store.append(ano)
        ncpt = 0
      else:
        ncpt += 1
        continue
      
      N_i = N_i + 1
      one = ano
      # print(N_i)
      if(N_i % 20 == 0):
        print("samples :", N_i)
      
      if N_i < N:
        continue
      else:
        break
    ## train N markov samples
    i = i + 1
    df_mar = pd.concat(df_store)
    temp_x = df_mar.drop('lettr', axis=1)
    temp_y = df_mar['lettr']

    if ker == "linear":
      temp_clf = SVC(kernel = ker)
    elif ker == "rbf":
      temp_clf = SVC(kernel = ker, random_state = 0)
    elif ker == "sigmoid":
      temp_clf = SVC(kernel = ker)
    elif ker == "poly":
      temp_clf = SVC(kernel = ker, degree = deg)

    temp_clf.fit(temp_x, temp_y)

    if i < k:
      continue
    else:
      break
  return temp_clf

In [ ]:
kt_clf = kt_svm(train, 100, 1, 1.2, 10, "linear")

samples : 20
samples : 40
samples : 60
samples : 80
samples : 100


In [ ]:
test_x = test.drop('lettr', axis=1)
test_y = test['lettr']

In [ ]:
kt_pred = kt_clf.predict(test_x)
print(classification_report(test_y, kt_pred))

              precision    recall  f1-score   support

           A       0.60      0.75      0.67       156
           B       0.22      0.39      0.28       136
           C       0.15      0.06      0.09       142
           D       0.00      0.00      0.00       167
           E       0.00      0.00      0.00       152
           F       0.22      0.54      0.31       153
           G       0.13      0.63      0.21       164
           H       0.00      0.00      0.00       151
           I       0.58      0.78      0.66       165
           J       1.00      0.23      0.37       148
           K       0.00      0.00      0.00       146
           L       0.87      0.74      0.80       157
           M       0.36      0.41      0.38       144
           N       0.47      0.04      0.08       166
           O       0.28      0.06      0.11       139
           P       0.00      0.00      0.00       168
           Q       0.26      0.53      0.35       168
           R       0.32    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
